<a href="https://colab.research.google.com/github/yusnivtr/Natural-Language-Processing-HCMUS/blob/main/skipgram_cbow_questions.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Word embedding and one-hot encoding







## One-hot encoding

> One-hot encoding is the process of turning categorical factors into a numerical structure that machine learning algorithms can readily process. It functions by representing each category in a feature as a binary vector of 1s and 0s, with the vector's size equivalent to the number of potential categories.

In [41]:
data = ['cold', 'cold', 'warm', 'cold', 'hot', 'hot', 'warm', 'cold', 'warm', 'hot']

### One-hot integer encoding

In [42]:
import numpy as np
from sklearn.preprocessing import LabelEncoder

label_encoder = LabelEncoder()
integer_encoded = label_encoder.fit_transform(np.array(data))

print(data)
print(integer_encoded)

['cold', 'cold', 'warm', 'cold', 'hot', 'hot', 'warm', 'cold', 'warm', 'hot']
[0 0 2 0 1 1 2 0 2 1]


### One-hot binary encoding

In [43]:
import numpy as np
from sklearn.preprocessing import OneHotEncoder
one_hot_encoder = OneHotEncoder()
integer_encoded = integer_encoded.reshape(len(integer_encoded), 1)
onehot_encoded_data = one_hot_encoder.fit_transform(integer_encoded)

print(data)
print(onehot_encoded_data)

['cold', 'cold', 'warm', 'cold', 'hot', 'hot', 'warm', 'cold', 'warm', 'hot']
  (np.int32(0), np.int32(0))	1.0
  (np.int32(1), np.int32(0))	1.0
  (np.int32(2), np.int32(2))	1.0
  (np.int32(3), np.int32(0))	1.0
  (np.int32(4), np.int32(1))	1.0
  (np.int32(5), np.int32(1))	1.0
  (np.int32(6), np.int32(2))	1.0
  (np.int32(7), np.int32(0))	1.0
  (np.int32(8), np.int32(2))	1.0
  (np.int32(9), np.int32(1))	1.0


## Problem 1
What are the limitations of one-hot encoding?

- Không thể nắm bắt mối quan hệ ngữ nghĩa giữa các từ. Trong biểu diễn one-hot, sự khác biệt giữa "mèo" và "chó" cũng giống như sự khác biệt giữa "mèo" và "giường".
- Đây là một kiểu biểu diễn thưa thớt (sparse representation) và có số chiều lớn (high-dimensional), bằng với kích thước của từ vựng |V|. Điều này có thể dẫn đến vấn đề về hiệu suất tính toán.
- Không linh hoạt trong việc xử lý các từ mới. Khi gặp một từ mới, cần phải gán cho nó một chỉ mục mới, điều này có thể làm thay đổi kích thước của biểu diễn và gây ra vấn đề cho các hệ thống NLP hiện có.
- Hầu như không chứa bất kỳ thông tin ngữ nghĩa nào về các từ, mà chỉ đơn giản là phân biệt chúng với nhau.
- Khi được dùng để biểu diễn câu hoặc tài liệu (dưới dạng tổng vector one-hot), phương pháp này bỏ qua hoàn toàn thứ tự của các từ. Do đó, các câu hoặc tài liệu khác nhau có thể có cùng một biểu diễn nếu chúng chứa cùng một tập hợp các từ

## Word embedding

ELI5 for word embeddings
> The word embeddings can be thought of as a child’s understanding of the words. Initially, the word embeddings are randomly initialized and they don’t make any sense, just like the baby has no understanding of different words. It’s only after the model has started getting trained, the word vectors/embeddings start to capture the meaning of the words, just like the baby hears and learns different words."

In [44]:
import torch
from torch import nn
from torch.nn import functional as F

In [45]:
import pandas as pd

corpus = [
    'This is the first document.',
    'This document is the second document.',
    'And this is the third one.',
    'Is this the first document?',
]

### Unigram transformation

In [46]:
from nltk import ngrams
from typing import List

def ngrams_transform(document: List[str],
                     n_gram: int) -> List[str]:
    """
    N-grams transformations for a given text

    Args:
    document (List[str]) -- The document to-be-processed
    n_gram   (int)       -- Number of grams

    Returns:
    A list of string after n-grams processed
    """

    n_grams_list = []
    for sentence in document:
        tokens = sentence.split()
        tokens = [token.lower() for token in tokens]
        n_grams = ngrams(tokens, n_gram)
        n_grams_list.extend([' '.join(gram) for gram in n_grams])
    return n_grams_list


In [47]:
n_grams_list = ngrams_transform(corpus,
                                n_gram=1)
n_grams_list


['this',
 'is',
 'the',
 'first',
 'document.',
 'this',
 'document',
 'is',
 'the',
 'second',
 'document.',
 'and',
 'this',
 'is',
 'the',
 'third',
 'one.',
 'is',
 'this',
 'the',
 'first',
 'document?']

In [48]:
# Integer label for the given corpus
label_encoder = LabelEncoder()
corpus_vector = label_encoder.fit_transform(np.array(n_grams_list))

# Tensorize the input vector
example_text_tensor = torch.Tensor(corpus_vector).to(dtype=torch.long)
print(f"Example text tensor: {example_text_tensor}")
print(f"Shape of example text tensor: {example_text_tensor.shape}")

Example text tensor: tensor([10,  5,  8,  4,  2, 10,  1,  5,  8,  7,  2,  0, 10,  5,  8,  9,  6,  5,
        10,  8,  4,  3])
Shape of example text tensor: torch.Size([22])


### Create an example for embedding function to map from a word dimension to a lower dimensional space

In [49]:
num_vocab = 22 # number of vocabulary
num_dimension = 50 # dimensional embeddings

# Declare the mapping function
example_embedding_function = nn.Embedding(num_vocab, num_dimension)

In [50]:
example_output_tensor = example_embedding_function(example_text_tensor)
print(f"Embedding shape: {example_output_tensor.shape}")

Embedding shape: torch.Size([22, 50])


# Word2vec


* Word2vec is a **class of models** that represents a word in a large text corpus as a vector in n-dimensional space(or n-dimensional feature space) bringing similar words closer to each other.



* Word2vec is a simple yet popular model to construct representating embedding for words from a representation space to a much lower dimensional space (compared to the respective number of words in a dictionary).



* Word2Vec has two neural network-based variants, which are:

    * Continuous Bag of Words (CBOW)
    * Skip-gram.
![](https://kavita-ganesan.com/wp-content/uploads/skipgram-vs-cbow-continuous-bag-of-words-word2vec-word-representation-2048x1075.png)


## Continuous Bag of words (CBOW)

* The Continuous Bag-of-Words model (CBOW) is frequently used in NLP deep learning. It is a model that tries to predict words given the context of a few words before and a few words after the target word. This is distinct from language modeling, since CBOW is not sequential and does not have to be probabilistic. Typically, CBOW is used to quickly train word embeddings, and these embeddings are used to initialize the embeddings of some more complicated model. Usually, this is referred to as pretraining embeddings. It almost always helps performance a couple of percent.

* CBOW is modelled as follows:
    * Given a target word $w_i$ and an $N$ context window on each side, $w_{i-1}, \cdots, w_{i-N}$ and $w_{i+1},\cdots, w_{i+N}$, referring to all context words collectively as $C$.

    * CBOW tries to minimize the objective function:

$$
-\log p(w_i|C) = -\log\text{Softmax}\left(A\left(\sum_{w\in C}q_w\right)+b\right)
$$

where $q_w$ is the embedding of word $w$.

In [51]:
# N = 2 according to the definition
CONTEXT_SIZE = 2

corpus = """We are about to study the idea of a computational process.
Computational processes are abstract beings that inhabit computers.
As they evolve, processes manipulate other abstract things called data.
The evolution of a process is directed by a pattern of rules
called a program. People create programs to direct processes. In effect,
we conjure the spirits of the computer with our spells."""

corpus = corpus.split()
len(corpus)

62

### Create an integer mapping

In [52]:
vocab = set(corpus)
vocab_size = len(vocab)

# Integer word mapping
word_to_idx = {word: i for i, word in enumerate(vocab)}
word_to_idx

{'evolution': 0,
 'rules': 1,
 'computer': 2,
 'spells.': 3,
 'idea': 4,
 'directed': 5,
 'The': 6,
 'As': 7,
 'about': 8,
 'direct': 9,
 'create': 10,
 'computers.': 11,
 'processes.': 12,
 'spirits': 13,
 'Computational': 14,
 'conjure': 15,
 'computational': 16,
 'we': 17,
 'that': 18,
 'programs': 19,
 'the': 20,
 'with': 21,
 'are': 22,
 'inhabit': 23,
 'In': 24,
 'to': 25,
 'called': 26,
 'process.': 27,
 'data.': 28,
 'pattern': 29,
 'is': 30,
 'a': 31,
 'process': 32,
 'We': 33,
 'they': 34,
 'beings': 35,
 'program.': 36,
 'things': 37,
 'by': 38,
 'People': 39,
 'effect,': 40,
 'our': 41,
 'processes': 42,
 'of': 43,
 'abstract': 44,
 'evolve,': 45,
 'study': 46,
 'other': 47,
 'manipulate': 48}

### Build context according to the given corpus

In [53]:
data = []

for i in range(CONTEXT_SIZE, len(corpus) - CONTEXT_SIZE):
    context = (
        [corpus[i - j - 1] for j in range(CONTEXT_SIZE)]
        + [corpus[i + j + 1] for j in range(CONTEXT_SIZE)]
    )
    target = corpus[i]
    data.append((context, target))

data

[(['are', 'We', 'to', 'study'], 'about'),
 (['about', 'are', 'study', 'the'], 'to'),
 (['to', 'about', 'the', 'idea'], 'study'),
 (['study', 'to', 'idea', 'of'], 'the'),
 (['the', 'study', 'of', 'a'], 'idea'),
 (['idea', 'the', 'a', 'computational'], 'of'),
 (['of', 'idea', 'computational', 'process.'], 'a'),
 (['a', 'of', 'process.', 'Computational'], 'computational'),
 (['computational', 'a', 'Computational', 'processes'], 'process.'),
 (['process.', 'computational', 'processes', 'are'], 'Computational'),
 (['Computational', 'process.', 'are', 'abstract'], 'processes'),
 (['processes', 'Computational', 'abstract', 'beings'], 'are'),
 (['are', 'processes', 'beings', 'that'], 'abstract'),
 (['abstract', 'are', 'that', 'inhabit'], 'beings'),
 (['beings', 'abstract', 'inhabit', 'computers.'], 'that'),
 (['that', 'beings', 'computers.', 'As'], 'inhabit'),
 (['inhabit', 'that', 'As', 'they'], 'computers.'),
 (['computers.', 'inhabit', 'they', 'evolve,'], 'As'),
 (['As', 'computers.', 'evol

### Problem 2
Name at least 2 limitations at this context construction step? Explain your answers.

**1. Mất dữ liệu ở phần đầu và cuối corpus**

Do vòng lặp chỉ chạy từ `CONTEXT_SIZE` đến `len(corpus) - CONTEXT_SIZE`, các phần tử ở đầu và cuối corpus (trong phạm vi `CONTEXT_SIZE`) sẽ không bao giờ được chọn làm target. Ví dụ, nếu `CONTEXT_SIZE = 2`, các từ ở vị trí 0, 1 và 2 phần tử cuối cùng sẽ bị loại bỏ. Điều này làm giảm lượng dữ liệu huấn luyện, đặc biệt nghiêm trọng nếu corpus ngắn hoặc `CONTEXT_SIZE` lớn.

**2. Kết hợp cả ngữ cảnh trước và sau có thể gây nhiễu thông tin**

Việc trộn cả từ trước và sau vị trí mục tiêu (target) vào context có thể không phù hợp với các mô hình yêu cầu thứ tự nghiêm ngặt (ví dụ: mô hình dự đoán từ tiếp theo chỉ dựa trên các từ trước). Hơn nữa, việc bao gồm cả từ phía sau có thể  tiết lộ thông tin về từ mục tiêu, dẫn đến overfitting (mô hình học vẹt thay vì hiểu bối cảnh thực sự).

### Vectorize context

In [54]:
tmp = np.array([item for item in vocab])
encoder = LabelEncoder()
idx = encoder.fit_transform(tmp)
word_to_idx = dict(zip(tmp, idx))

def make_context_vector(context: List[str],
                        word_to_idx: dict) -> torch.Tensor:
    """
    Function to map a word context vector into a torch tensor

    Args:
    context (List[str]) -- A context (including individual n-grams tokens)
    word_to_idx (dict)  -- A functionto map a word into its respective integer

    Returns:
    A pytorch tensor including a list of mapped word

    Example:
    ['are', 'We', 'to', 'study'] --> tensor([40, 22, 27, 47])
    """

    ### START YOUR CODE HERE ###
    vector = [word_to_idx[word] for word in context]
    vector = torch.tensor(vector, dtype=torch.long)
    return vector


    ### END YOUR CODE HERE ###
    
# Functional test
print("Example sample: ", data[0][0])
make_context_vector(data[0][0], word_to_idx)


Example sample:  ['are', 'We', 'to', 'study']


tensor([ 9,  5, 46, 41])

In [55]:
# Functional test
print("Example sample: ", data[0][0])
make_context_vector(data[0][0], word_to_idx)

Example sample:  ['are', 'We', 'to', 'study']


tensor([ 9,  5, 46, 41])

### CBOW model implementation

In [56]:
class CBOW(nn.Module):
    def __init__(self,
                 vocab_size: int,
                 embed_dim: int) -> None:
        """
        Model constructor
        """
        super().__init__()

        self.vocab_size = vocab_size
        self.embed_dim = embed_dim

        self.embedding_layer = nn.Embedding(vocab_size, embed_dim)
        self.linear_layer = nn.Linear(embed_dim, vocab_size)

        # Neural weight initialization
        nn.init.xavier_normal_(self.embedding_layer.weight)
        nn.init.xavier_normal_(self.linear_layer.weight)

    def forward(self, inputs):
        """
        Function to conduct forward passing
        """
        embedding = self.embedding_layer(inputs)
        embedding = torch.sum(embedding, dim=1)
        output = self.linear_layer(embedding)
        output_softmax = F.log_softmax(output, dim=1)
        return output_softmax

In [57]:
cbow_model = CBOW(vocab_size=vocab_size,
                  embed_dim=10)

# Enable gradient for model training
cbow_model.train()
cbow_model

CBOW(
  (embedding_layer): Embedding(49, 10)
  (linear_layer): Linear(in_features=10, out_features=49, bias=True)
)

### Train

#### Hyperparameters and training configuration

In [58]:
num_epochs: int = 5
learning_rate: float = 5e-2
optimizer: torch.optim = torch.optim.Adam(cbow_model.parameters(),
                                          lr=learning_rate)

loss_function = nn.NLLLoss()

#### Training phase

In [59]:
for epoch in range(1, num_epochs + 1):
    print(f"Epoch {epoch}/{num_epochs}")

    # Construct input and target tensor
    input_vector, target_vector = torch.tensor(make_context_vector(data[0][0], word_to_idx)), torch.tensor(word_to_idx[data[0][1]])
    input_vector = input_vector.unsqueeze(0)
    target_vector = target_vector.unsqueeze(0)

    # Join whole data into 1 tensor set
    for idx in range(1, len(data)):
        input_tensor = torch.tensor(make_context_vector(data[idx][0], word_to_idx)).unsqueeze(0)
        target_tensor = torch.tensor(word_to_idx[data[idx][1]]).unsqueeze(0)
        torch.cat((input_vector, input_tensor), 0)
        torch.cat((target_vector, target_tensor), 0)

    # Zero out the gradients from the old instance to avoid tensor accumulation
    cbow_model.zero_grad()

    # Forward passing
    log_probabilities = cbow_model(input_vector)

    # Evaluate loss
    loss = loss_function(log_probabilities, target_vector)

    # Backpropagation
    loss.backward()

    # Update the gradient according to the optimization algorithm
    optimizer.step()

    # Get loss values
    epoch_loss = loss.item()
    print("Loss:", epoch_loss)

Epoch 1/5
Loss: 3.84928560256958
Epoch 2/5
Loss: 3.1683542728424072
Epoch 3/5
Loss: 2.5093705654144287
Epoch 4/5
Loss: 1.7346961498260498
Epoch 5/5
Loss: 0.8970910310745239


<ipython-input-59-97ca5e4a0b37>:5: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  input_vector, target_vector = torch.tensor(make_context_vector(data[0][0], word_to_idx)), torch.tensor(word_to_idx[data[0][1]])
<ipython-input-59-97ca5e4a0b37>:11: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  input_tensor = torch.tensor(make_context_vector(data[idx][0], word_to_idx)).unsqueeze(0)


#### Inference

In [107]:
with torch.no_grad(): # No gradient update in inference
    context = ['In', 'processes.', 'we', 'conjure']

    # Vectorize input from text to numeric type
    input_tensor = torch.tensor(make_context_vector(context, word_to_idx)).unsqueeze(0)
    # Model makes prediction
    output_tensor = cbow_model(input_tensor)
    # Get the item id with the highest probability
    prediction = torch.argmax(output_tensor).detach().tolist()
    # Query the respective word from the given item id
    key_list = list(word_to_idx.keys())
    prediction = key_list[prediction]

    print("Context:", context)
    print("Prediction:", prediction)

tensor([[ 2, 35, 47, 16]])
torch.Size([1, 4])
Context: ['In', 'processes.', 'we', 'conjure']
Prediction: As


<ipython-input-107-a022f23a6ee1>:5: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  input_tensor = torch.tensor(make_context_vector(context, word_to_idx)).unsqueeze(0)


In [118]:
input_vector, target_vector = torch.tensor(make_context_vector(data[0][0], word_to_idx)), torch.tensor(word_to_idx[data[0][1]])
# input_vector.unsqueeze(-1)
# target_vector = target_vector.unsqueeze(0)
input_tensor.shape

<ipython-input-118-f872ab30713d>:1: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  input_vector, target_vector = torch.tensor(make_context_vector(data[0][0], word_to_idx)), torch.tensor(word_to_idx[data[0][1]])


torch.Size([1, 4])

## Skip-gram

<center>
<img src="https://machinelearningcoban.com/tabml_book/_images/word2vec2.png">
</center>

- Skip gram is based on the distributional hypothesis where words with similar distribution is considered to have similar meanings. Researchers of skip gram suggested a model with less parameters along with the novel methods to make optimization step more efficient.

- Vanilla SkipGram model:

<center>
<img src="https://d3i71xaburhd42.cloudfront.net/a1d083c872e848787cb572a73d97f2c24947a374/5-Figure1-1.png" scale=70%>
</center>

- Main idea is to optimize model so that if it is queried with a word, it should correctly guess all the context (context = 2 in the figure) words. That is,
$$
y=\sigma(Ux)
$$
    - where $x$, $y$ are one-hot encoded word vector, $U$ is the embedding matrix, and $\sigma(\cdot)$ is the softmax function.

With the same dataset, training set for skip gram can be much larger than that of NPLM since it can have $2c$ samples $\left(w_t:w_{t-c}, ...,w_t:w_{t-1},w_t:w_{t+1},...,w_{t+c}\right)$ while other n-gram based models have one $\left((w_{t-c},...w_{t-1},w_{t+1},...,w_{t+c}):w_t\right)$.

In [61]:
corpus = """We are about to study the idea of a computational process.
Computational processes are abstract beings that inhabit computers.
As they evolve, processes manipulate other abstract things called data.
The evolution of a process is directed by a pattern of rules
called a program. People create programs to direct processes. In effect,
we conjure the spirits of the computer with our spells."""

In [69]:
def create_vocab(corpus):
    words = corpus.split()
    return set(words)

vocab = create_vocab(corpus)

{'As',
 'Computational',
 'In',
 'People',
 'The',
 'We',
 'a',
 'about',
 'abstract',
 'are',
 'beings',
 'by',
 'called',
 'computational',
 'computer',
 'computers.',
 'conjure',
 'create',
 'data.',
 'direct',
 'directed',
 'effect,',
 'evolution',
 'evolve,',
 'idea',
 'inhabit',
 'is',
 'manipulate',
 'of',
 'other',
 'our',
 'pattern',
 'process',
 'process.',
 'processes',
 'processes.',
 'program.',
 'programs',
 'rules',
 'spells.',
 'spirits',
 'study',
 'that',
 'the',
 'they',
 'things',
 'to',
 'we',
 'with'}

In [ ]:
class SkipGramModel(nn.Module):
    def __init__(self,
                 vocab_size: int,
                 embed_dim: int) -> None:
        """
        Model construction
        """
        super().__init__()
        self.vocab_size = vocab_size
        self.embed_dim = embed_dim

        ### START YOUR CODE HERE ###
        # Declare embedding function u and v
        # with given vocab size and embed dim using nn.Embedding
        self.v_embedding_layer = nn.Embedding(vocab_size,embed_dim)
        self.u_embedding_layer = nn.Embedding(vocab_size,embed_dim)

        # Network weight initialization with Xavier initialization
        nn.init.xavier_normal_(self.v_embedding_layer.weight)
        nn.init.xavier_normal_(self.u_embedding_layer.weight)

        ### END YOUR CODE HERE ###

    def forward(self, center_words, context):
        """
        Function to perform forward passing
        """
        v_embedding = self.v_embedding_layer(center_words)
        u_embedding = self.u_embedding_layer(context)

        score = torch.mul(v_embedding, u_embedding)
        score = torch.sum(score, dim=-1)
        log_score = F.logsigmoid(score)
        return log_score

In [101]:
skipgram_model = SkipGramModel(vocab_size=vocab_size,
                               embed_dim=128)

skipgram_model.train()
skipgram_model

SkipGramModel(
  (v_embedding_layer): Embedding(49, 128)
  (u_embedding_layer): Embedding(49, 128)
)

### Prepare training data to match the format of SkipGram model

In [102]:
def gather_training_data(corpus,
                         word_to_idx: dict,
                         context_size: int):
    """
    This function is to transform the given corpus
    into the correct format for SkipGram to serve as its input
    """

    training_data = []
    all_vocab_indices = list(range(len(word_to_idx)))

    split_text = corpus.split('\n')

    # For each sentence
    for sentence in split_text:
        indices = []
        indices = [word_to_idx[word] for word in sentence.split(' ')]

        # For each word treated as center word
        for center_word_pos in range(len(indices)):

            # For each window  position
            for w in range(-context_size, context_size+1):
                context_word_pos = center_word_pos + w

                # Make sure we dont jump out of the sentence
                if context_word_pos < 0 or context_word_pos >= len(indices) or center_word_pos == context_word_pos:
                    continue

                context_word_idx = indices[context_word_pos]
                center_word_idx  = indices[center_word_pos]

                # Same words might be present in the close vicinity of each other. we want to avoid such cases
                if center_word_idx == context_word_idx:
                    continue

                training_data.append([center_word_idx, context_word_idx])

    return training_data

In [103]:
training_data = gather_training_data(corpus,
                                     word_to_idx,
                                     context_size=2)
training_data = torch.tensor(training_data).to(dtype=torch.long)
training_data

tensor([[ 5,  9],
        [ 5,  7],
        [ 9,  5],
        [ 9,  7],
        [ 9, 46],
        [ 7,  5],
        [ 7,  9],
        [ 7, 46],
        [ 7, 41],
        [46,  9],
        [46,  7],
        [46, 41],
        [46, 43],
        [41,  7],
        [41, 46],
        [41, 43],
        [41, 24],
        [43, 46],
        [43, 41],
        [43, 24],
        [43, 28],
        [24, 41],
        [24, 43],
        [24, 28],
        [24,  6],
        [28, 43],
        [28, 24],
        [28,  6],
        [28, 13],
        [ 6, 24],
        [ 6, 28],
        [ 6, 13],
        [ 6, 33],
        [13, 28],
        [13,  6],
        [13, 33],
        [33,  6],
        [33, 13],
        [ 1, 34],
        [ 1,  9],
        [34,  1],
        [34,  9],
        [34,  8],
        [ 9,  1],
        [ 9, 34],
        [ 9,  8],
        [ 9, 10],
        [ 8, 34],
        [ 8,  9],
        [ 8, 10],
        [ 8, 42],
        [10,  9],
        [10,  8],
        [10, 42],
        [10, 25],
        [4

### Hyperparamters and training configuration

In [104]:
num_epochs: int = 200
learning_rate: float = 5e-1
optimizer: torch.optim = torch.optim.SGD(skipgram_model.parameters(),
                                          lr=learning_rate)

### Training phase

In [132]:
from tqdm.auto import tqdm

for epoch in tqdm(range(num_epochs + 1)):
    """
    Adapt the given CBOW training code for SkipGram
    Following by the instruction comments, or you could do it on your own ;)
    """
    ### START YOUR CODE HERE ###
    print(f"Epoch {epoch}/{num_epochs}")
    # Construct input and target tensor
    inputs, targets = training_data[0][0], training_data[0][1]
    inputs = inputs.unsqueeze(0)
    targets = targets.unsqueeze(0)
    # Zero out the gradients from the old instance to avoid tensor accumulation
    optimizer.zero_grad()

    # Forward passing
    logsoftmax_prediction = skipgram_model(inputs, targets)

    # Evaluate loss (Negative log likelihood)
    loss = torch.mean(-1 * logsoftmax_prediction)

    # Backpropagation
    loss.backward()

    # Update the gradient according to the optimization algorithm
    optimizer.step()

    # Get loss values
    epoch_loss = loss.item()

    # Log result
    if epoch % 50 == 0:
        print(f"#Epoch {epoch}/{num_epochs}")
        print("Loss:", epoch_loss)

    ### END YOUR CODE HERE ###

  0%|          | 0/201 [00:00<?, ?it/s]

Epoch 0/200
#Epoch 0/200
Loss: 0.7464172840118408
Epoch 1/200
Epoch 2/200
Epoch 3/200
Epoch 4/200
Epoch 5/200
Epoch 6/200
Epoch 7/200
Epoch 8/200
Epoch 9/200
Epoch 10/200
Epoch 11/200
Epoch 12/200
Epoch 13/200
Epoch 14/200
Epoch 15/200
Epoch 16/200
Epoch 17/200
Epoch 18/200
Epoch 19/200
Epoch 20/200
Epoch 21/200
Epoch 22/200
Epoch 23/200
Epoch 24/200
Epoch 25/200
Epoch 26/200
Epoch 27/200
Epoch 28/200
Epoch 29/200
Epoch 30/200
Epoch 31/200
Epoch 32/200
Epoch 33/200
Epoch 34/200
Epoch 35/200
Epoch 36/200
Epoch 37/200
Epoch 38/200
Epoch 39/200
Epoch 40/200
Epoch 41/200
Epoch 42/200
Epoch 43/200
Epoch 44/200
Epoch 45/200
Epoch 46/200
Epoch 47/200
Epoch 48/200
Epoch 49/200
Epoch 50/200
#Epoch 50/200
Loss: 0.004693763330578804
Epoch 51/200
Epoch 52/200
Epoch 53/200
Epoch 54/200
Epoch 55/200
Epoch 56/200
Epoch 57/200
Epoch 58/200
Epoch 59/200
Epoch 60/200
Epoch 61/200
Epoch 62/200
Epoch 63/200
Epoch 64/200
Epoch 65/200
Epoch 66/200
Epoch 67/200
Epoch 68/200
Epoch 69/200
Epoch 70/200
Epoch 71

### Inference

In [133]:
with torch.no_grad():
    context = ['we']

    ### START YOUR CODE HERE ###
    # Based on the given inference code in the previous section, training code and the context
    # Implement the inference flow from the given context to an output word
    input = word_to_idx[context[0]]
    input_tensor = torch.tensor(input).unsqueeze(0)
    
    u_embedding_context = skipgram_model.u_embedding_layer(input_tensor)
    
    all_v_embeddings = skipgram_model.v_embedding_layer.weight
    
    
    similarity_scores = torch.matmul(all_v_embeddings,u_embedding_context.T)
    
    best_center_word_idx = torch.argmax(similarity_scores)

    key_list = list(word_to_idx.keys())
    prediction = key_list[best_center_word_idx]
    ### END YOUR CODE HERE ###
    print("Context:", context)
    print("Prediction:", prediction)

Context: ['we']
Prediction: directed


## Problem 3
What are the differences between CBOW and Skip-gram?


1.  **Mục tiêu Dự đoán (Prediction Goal):**
    *   **CBOW:** Dự đoán từ *mục tiêu* (target word) dựa trên các từ *ngữ cảnh* (context words) xung quanh nó. 
    *   **Skip-gram:** Dự đoán các từ *ngữ cảnh* xung quanh dựa trên một từ *mục tiêu* cho trước.

2.  **Đầu vào và Đầu ra (Input & Output):**
    *   **CBOW:**
        *   *Đầu vào:* Vectors của các từ ngữ cảnh (thường được kết hợp lại, ví dụ: lấy trung bình).
        *   *Đầu ra:* Phân phối xác suất trên toàn bộ từ vựng cho từ mục tiêu.
    *   **Skip-gram:**
        *   *Đầu vào:* Vector của từ mục tiêu.
        *   *Đầu ra:* *Nhiều* phân phối xác suất trên toàn bộ từ vựng, mỗi cái cho một vị trí từ ngữ cảnh.

3.  **Tốc độ Huấn luyện (Training Speed):**
    *   **CBOW:** Thường huấn luyện *nhanh hơn* Skip-gram. Lý do là CBOW cập nhật trọng số dựa trên việc dự đoán một từ duy nhất từ nhiều từ ngữ cảnh (ít tác vụ dự đoán hơn cho mỗi mẫu huấn luyện).
    *   **Skip-gram:** Thường huấn luyện *chậm hơn* CBOW. Do một phần là nó phải thực hiện nhiều dự đoán (cho mỗi từ ngữ cảnh) từ một từ mục tiêu duy nhất (nhiều tác vụ dự đoán hơn cho mỗi mẫu huấn luyện).

4.  **Hiệu quả với Dữ liệu và Từ hiếm (Performance with Data and Rare Words):**
    *   **CBOW:** Hoạt động tốt với các tập dữ liệu lớn. Việc lấy trung bình ngữ cảnh làm cho nó hiệu quả hơn với các từ xuất hiện thường xuyên (frequent words).
    *   **Skip-gram:** Hoạt động tốt ngay cả với các tập dữ liệu nhỏ hơn. Quan trọng hơn, nó có xu hướng học được các biểu diễn tốt hơn cho các từ xuất hiện *ít thường xuyên* (rare words) vì nó xem xét từng cặp (từ mục tiêu, từ ngữ cảnh) một cách riêng lẻ, không bị "pha loãng" bởi việc lấy trung bình.

5.  **Chất lượng Embedding (Embedding Quality):**
    *   **Skip-gram:** Thường được cho là tạo ra các word embeddings có chất lượng cao hơn một chút, đặc biệt là trong việc nắm bắt các mối quan hệ ngữ nghĩa và cú pháp tinh tế, nhất là đối với từ hiếm.
    *   **CBOW:** Vẫn tạo ra embeddings chất lượng tốt, đặc biệt hiệu quả về mặt tốc độ và bộ nhớ.

**Tóm laij:**

| Đặc điểm             | CBOW (Continuous Bag-of-Words)              | Skip-gram                                     |
| :------------------- | :------------------------------------------ | :-------------------------------------------- |
| **Mục tiêu**         | Dự đoán từ mục tiêu từ ngữ cảnh             | Dự đoán ngữ cảnh từ từ mục tiêu                |
| **Cách tiếp cận**    | Ngữ cảnh -> Từ mục tiêu                     | Từ mục tiêu -> Ngữ cảnh                        |
| **Tốc độ huấn luyện** | Nhanh hơn                                    | Chậm hơn                                     |
| **Từ hiếm**          | Kém hiệu quả hơn                            | Hiệu quả hơn                                 |
| **Tập dữ liệu lớn**  | Hiệu quả tốt                                | Vẫn tốt, nhưng chậm hơn                      |
| **Chất lượng**       | Tốt, nhanh hơn                              | Thường tốt hơn một chút, đặc biệt từ hiếm     |